### Example 1 - One chain

In [1]:
# manual grad
# y = x**2
# dy/dx = 2x
def dy_dx(x):
  return 2*x

dy_dx(3)

6

In [2]:
import torch

In [3]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [4]:
y = x**2
y

tensor(9., grad_fn=<PowBackward0>)

In [5]:
y.backward() # calculate the grad using grad_fn
x.grad

tensor(6.)

### Example 2 - Two chain

In [6]:
import math

# manual grad
# y = x**2
# z = sin(y)
# dy/dx = 2*x*cos(x**2)
def dz_dx(x):
    return 2 * x * math.cos(x**2)

dz_dx(4)

-7.661275842587077

In [7]:
x = torch.tensor(4.0, requires_grad=True)
x

tensor(4., requires_grad=True)

In [8]:
y = x ** 2
y

tensor(16., grad_fn=<PowBackward0>)

In [9]:
z = torch.sin(y)
z

tensor(-0.2879, grad_fn=<SinBackward0>)

In [10]:
z.backward()
x.grad

tensor(-7.6613)

In [11]:
# will not work, not a leaf Tensor
y.grad

C:\Users\Nova\AppData\Local\Temp\ipykernel_9232\3602219156.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  y.grad


### Example 3 - Perceptron

In [12]:
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # weight
b = torch.tensor(0.0)  # bias

print(w)
print(b)

tensor(1.)
tensor(0.)


In [13]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [14]:
# forward pass
z = w * x + b  # weighted sum (linear part)
y_pred = torch.sigmoid(z)  # predicted probability

# compute loss
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012)

In [15]:
# Manual rerivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

print(f"Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Gradient of loss w.r.t bias   (db): {dL_db}")

Gradient of loss w.r.t weight (dw): 6.691762447357178
Gradient of loss w.r.t bias   (db): 0.998770534992218


In [16]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

print(w)
print(b)

tensor(1., requires_grad=True)
tensor(0., requires_grad=True)


In [17]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [18]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [19]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [20]:
loss.backward()
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


In [21]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

In [22]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [23]:
y.backward()
x.grad

tensor([0.6667, 1.3333, 2.0000])

### Clearing grad

In [24]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [25]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [26]:
y.backward() # acumulate with previous grad
x.grad

tensor(4.)

In [27]:
x.grad.zero_() # clear previous grad
x.grad

tensor(0.)

### Disable gradient tracking

In [28]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [29]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [30]:
y.backward()
x.grad

tensor(4.)

In [31]:
# option 1 - requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad() or torch.inference_mode()

In [32]:
x.requires_grad_(False)
x

tensor(2.)

In [33]:
y1 = x ** 2
y1

tensor(4.)

In [34]:
# will not work, does not have a grad_fn
# y1.backward()

In [35]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [36]:
z = x.detach()
z

tensor(2.)

In [37]:
y2 = z ** 2
y2

tensor(4.)

In [38]:
# will not work, does not have a grad_fn
#y2.backward()

In [39]:
with torch.no_grad():
    y3 = z ** 2
y3

tensor(4.)

In [40]:
# will not work, does not have a grad_fn
#y3.backward()

In [41]:
with torch.inference_mode():
    y4 = z ** 2
y4

tensor(4.)

In [42]:
# will not work, does not have a grad_fn
#y4.backward()